In [1]:
# import necessry packages
import cv2               
import numpy as np
import os



In [11]:
# since we have 2 persons(folder name as person1 & person2)
subjects =  [""," PM Imran Khan", "Nawaz Sharif"]



In [12]:
# function to be called as we have to detect face in given image after training our model
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
   
    face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=6);
    
    if (len(faces) == 0):
        return None, None
    
   
    (x, y, w, h) = faces[0]
    
    return gray[y:y+w, x:x+h], faces[0]


In [13]:
def prepare_training_data(data_folder_path):
    
  
    dirs = os.listdir(data_folder_path)
    
    faces = []
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignore any non-relevant directories if any
        if not dir_name.startswith("person"):
            continue;
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        #format of dir name = slabel
        #, so removing letter 's' from dir_name will give us label
        label = int(dir_name.replace("person", ""))
        
        #build path of directory containin images for current subject subject
        #sample subject_dir_path = "training-data/s1"
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
   
        for image_name in subject_images_names:
            
            
            if image_name.startswith("."):
                continue;
            
           
            image_path = subject_dir_path + "/" + image_name

            
            image = cv2.imread(image_path)
            
            cv2.imshow("Training on image...", cv2.resize(image, (500,500)))
            cv2.waitKey(100)
            
            face, rect = detect_face(image)
            
           
            if face is not None:
                faces.append(face)
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels



In [14]:

print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))



Preparing data...
Data prepared
Total faces:  57
Total labels:  57


In [15]:
#create our LBPH face recognizer 
face_recognizer = cv2.face.LBPHFaceRecognizer_create()



In [16]:
#train our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))



In [17]:
# draw bounding box to detected/recognize faces
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+w), (0, 255, 0), 2)
    
# draw text (name) of predected output     
def draw_text(img, text, x, y):
    
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)





In [18]:
def predict(test_img):
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label, confidence = face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label]
    #confi = conf[confidence]
   # label_text = subjects[int(confidence)]
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-2)
    
    return img


In [19]:


print("Predicting images...")

#load test images
test_img1 = cv2.imread("test-data/test1.jpg")
test_img2 = cv2.imread("test-data/test2.jpeg")

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")

#display both images
cv2.imshow(subjects[1], cv2.resize(predicted_img1, (400, 500)))
cv2.imshow(subjects[2], cv2.resize(predicted_img2, (400, 600)))
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.destroyAllWindows()



Predicting images...
Prediction complete
